# This notebook illustrates the workflow for converting field data from another group in .xlsx file format to .csv files, keeping only the necessary data.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Set some directories. Here we use absolute directories. 
cwd = 'c:\\Users\\m1865\\Desktop\\Ticino'
cwd_Field = cwd + '\\FieldData'
cwd_Field_Data2 = cwd_Field + '\\Field Dataset 2'
cwd_Field_Data2_CSV = cwd_Field_Data2 + "\\CSV"

### Purge the raw Excel file

In [3]:
# Since our Excel file contains multiple sheets, we need to get the names of sheets firstly. 
xlsx = pd.ExcelFile(cwd_Field_Data2 + '\\Dati plot forestali Parco Ticino completo.xlsx')
xlsx.sheet_names

['Querce caducifoglie',
 'Acero-fra, Orno-ost',
 'Castagneti',
 'Pinete con lat',
 'Lat alloctone',
 'Lat meso con conifere',
 'TEMPLATE']

In [4]:
# Combine these sheets into a single one! 
xlsx_Field = pd.DataFrame()
for sheet in xlsx.sheet_names:
    if sheet != 'TEMPLATE':
        temp_df = pd.read_excel(cwd_Field_Data2 + '\\Dati plot forestali Parco Ticino completo.xlsx', sheet_name=sheet)
        xlsx_Field = pd.concat([xlsx_Field, temp_df], ignore_index=True)
xlsx_Field

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Tipo forestale:,NaN,Querce caducifoglie (colore rosso),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,ha factor,NaN,NaN,NaN,NaN,NaN
3,Radius [m],10,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
972,40,9,casa,25.5,NaN,NaN
973,40,10,prse,10.5,NaN,NaN
974,40,11,quro,41.5,NaN,NaN
975,40,12,quro,48,NaN,NaN


In [5]:
# By observing our concatenated dataframe, we only need to keep the first 3 columns. 
# Drop those columns! 
xlsx_Field = xlsx_Field.drop(xlsx_Field.iloc[:,np.r_[1,np.arange(3,6,1)]], axis = 1)
xlsx_Field

,Unnamed: 0,Unnamed: 2
0,Tipo forestale:,Querce caducifoglie (colore rosso)
1,NaN,NaN
2,ha factor,NaN
3,Radius [m],NaN
4,NaN,NaN
...,...,...
972,40,casa
973,40,prse
974,40,quro
975,40,quro


In [6]:
# Now we need to drop the useless rows. If the first column of a row is not a numeric value, then it will be dropped. 
df_Field = xlsx_Field.copy()
for row in range(0,xlsx_Field.shape[0]):
    # Get the index label of the current row! 
    temp_label = xlsx_Field.iloc[row].name
    if str(xlsx_Field.iloc[row,0]).isnumeric(): # Note: we have to convert the value to str first, otherwise we can't apply "isnumeric()" function. 
        continue
    else: 
        # Drop the row based on the label! 
        df_Field = df_Field.drop(index=temp_label, axis = 0)
# Reset index
df_Field = df_Field.reset_index(drop = True)
# Assign column names
df_Field.columns = ["Plot", "Species"]
# Finally we get our clean dataframe! 
df_Field

,Plot,Species
0,1,quru
1,1,pisy
2,1,quru
3,1,quru
4,1,quru
...,...,...
834,40,casa
835,40,prse
836,40,quro
837,40,quro


In [7]:
# Now we group by to aggregate some values
df_Field_Count = df_Field.groupby(['Plot', 'Species'], as_index = False).size()
df_Field_Count = df_Field_Count.rename(columns = {'size': 'Quantity'})
df_Field_Count

,Plot,Species,Quantity
0,1,pisy,2
1,1,quru,7
2,2,pisy,1
3,2,prse,12
4,2,quro,5
...,...,...,...
147,39,pisy,7
148,40,casa,2
149,40,pisy,2
150,40,prse,6


### Save this refined dataframe as .csv file. 

In [8]:
df_Field_Count.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 All with Species.csv", index = False)

### Merge the geographical coordinates of the plots and also save them as .csv files

In [9]:
# Read another Excel file which contains all the geographical coordinates of the field plots! 
df_Field_All_LatLong = pd.read_csv(cwd_Field_Data2_CSV + "\\FieldData2 All LatLong.csv")
df_Field_All_UTM = pd.read_csv(cwd_Field_Data2_CSV + "\\FieldData2 All UTM.csv")

In [10]:
df_FieldMerge_All_LatLong = pd.merge(df_Field_Count,df_Field_All_LatLong,on = 'Plot')
df_FieldMerge_All_UTM = pd.merge(df_Field_Count,df_Field_All_UTM,on = 'Plot')
df_FieldMerge_All_UTM

,Plot,Species,Quantity,UTM_X,UTM_Y
0,1,pisy,2,481149.209730,5.062108e+06
1,1,quru,7,481149.209730,5.062108e+06
2,2,pisy,1,480551.550682,5.061058e+06
3,2,prse,12,480551.550682,5.061058e+06
4,2,quro,5,480551.550682,5.061058e+06
...,...,...,...,...,...
147,39,pisy,7,476404.258874,5.064511e+06
148,40,casa,2,479556.832180,5.056529e+06
149,40,pisy,2,479556.832180,5.056529e+06
150,40,prse,6,479556.832180,5.056529e+06


In [11]:
df_FieldMerge_All_LatLong.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 All LatLong with Species.csv")
df_FieldMerge_All_UTM.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 All UTM with Species.csv")

### Now we filter out the plots outside our RoI

In [12]:
# Read the filtered out .csv file
df_Field_Valid = pd.read_csv(cwd_Field_Data2_CSV + "\\FieldData2 Valid LatLong.csv")
df_Field_Valid.tail()

,Plot,Lat,Long
28,35,45.743313,8.627677
29,37,45.665895,8.682949
30,38,45.686724,8.756250
31,39,45.733749,8.696724
32,40,45.662013,8.737580


In [13]:
# Get an array of the numbers of the plots inside our RoI
array_Valid = df_Field_Valid['Plot'].unique()
array_Valid

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 21,
       22, 23, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 37, 38, 39, 40],
      dtype=int64)

In [14]:
# Mask the invalid plots
mask_Valid = df_Field_Count['Plot'].isin(array_Valid)
df_Field_Count_Valid = df_Field_Count[mask_Valid].reset_index(drop = True)
df_Field_Count_Valid

,Plot,Species,Quantity
0,1,pisy,2
1,1,quru,7
2,2,pisy,1
3,2,prse,12
4,2,quro,5
...,...,...,...
121,39,pisy,7
122,40,casa,2
123,40,pisy,2
124,40,prse,6


In [15]:
# Mask the invalid plots
mask_Valid = df_FieldMerge_All_LatLong['Plot'].isin(array_Valid)
df_Field_Valid_LatLong = df_FieldMerge_All_LatLong[mask_Valid].reset_index(drop = True)
df_Field_Valid_LatLong

,Plot,Species,Quantity,Lat,Long
0,1,pisy,2,45.712265,8.757804
1,1,quru,7,45.712265,8.757804
2,2,pisy,1,45.702801,8.750167
3,2,prse,12,45.702801,8.750167
4,2,quro,5,45.702801,8.750167
...,...,...,...,...,...
121,39,pisy,7,45.733749,8.696724
122,40,casa,2,45.662013,8.737580
123,40,pisy,2,45.662013,8.737580
124,40,prse,6,45.662013,8.737580


In [16]:
# Mask the invalid plots
mask_Valid = df_FieldMerge_All_UTM['Plot'].isin(array_Valid)
df_Field_Valid_UTM = df_FieldMerge_All_UTM[mask_Valid].reset_index(drop = True)
df_Field_Valid_UTM

,Plot,Species,Quantity,UTM_X,UTM_Y
0,1,pisy,2,481149.209730,5.062108e+06
1,1,quru,7,481149.209730,5.062108e+06
2,2,pisy,1,480551.550682,5.061058e+06
3,2,prse,12,480551.550682,5.061058e+06
4,2,quro,5,480551.550682,5.061058e+06
...,...,...,...,...,...
121,39,pisy,7,476404.258874,5.064511e+06
122,40,casa,2,479556.832180,5.056529e+06
123,40,pisy,2,479556.832180,5.056529e+06
124,40,prse,6,479556.832180,5.056529e+06


### Save these filtered out dataframes as .csv files

In [17]:
df_Field_Count_Valid.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 Valid with Species.csv", index = False)
df_Field_Valid_UTM.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 Valid UTM with Species.csv", index = False)
df_Field_Valid_LatLong.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 Valid LatLong with Species.csv", index = False)